Initial draft of data specifications for cell classification task.

## Summary

Files must be zip containers (filenames will end in '.zip') and include:
- X.npy array with dimensions (1, y, x, c); raw data
- y.npy array with dimensions (1, y, x, 1); instance labels
- channel_names.json
- classes/cell_type.json

This notebook will help create each component of the file and save them in the correct output format. This notebook also provides an example of how to extract the file contents after annotation using the python zipfile library.